In [1]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 87 kB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 18.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 43.2 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 


In [3]:
!pip install pyngrok
!pyngrok authtoken 29JlwqUGLZxPpC0QfilJEO8AwsW_xxCAFaGtukqQ8krP2CFd

     |████████████████████████████████| 745 kB 7.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=c581fb1b1520b6e993b7b01ece7f2daa7b6e7c92c744987f2a4e61850e0b0a41
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [5]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("7EViQGhjLoXD6HnX+H2llWpD5+bO3sEXJZ/E85+X6YCwNud3ukbdZryinFuVzq0yoLlycjYLx4aqu/DwLhgiuCuzmWfN9Rqt93/4DbuaELQoORWV4OSAA/POyr7vkL+Ur96YcMEguV51FyKT5oVfoAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("4c7bc83333634370cd348be01acb374d")

In [6]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [7]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")

analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )

In [8]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message
}

In [9]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質")
        )

In [10]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://74b5-35-197-24-247.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[]}


127.0.0.1 - - [18/May/2022 05:34:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 05:34:56] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106405846521","text":"@123"},"webhookEventId":"01G3ATJZ67TJ48G217XTQTX6CE","deliveryContext":{"isRedelivery":false},"timestamp":1652852095893,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"eeb27b01afc54db195e6aef67316a11c","mode":"active"}]}


127.0.0.1 - - [18/May/2022 05:35:12] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106406952097","text":"測試"},"webhookEventId":"01G3ATKEF2CBFHGVT02RDA7W7E","deliveryContext":{"isRedelivery":false},"timestamp":1652852111441,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"4b324e98594d498b9341087e13215219","mode":"active"}]}


127.0.0.1 - - [18/May/2022 05:35:25] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16106407900284","text":"@分析的本質"},"webhookEventId":"01G3ATKV4NKQW1NS5GYQHAME0A","deliveryContext":{"isRedelivery":false},"timestamp":1652852124681,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"db7d05221aa4473eb5eb8537583a62b5","mode":"active"}]}
